## Clean Old Demo

In [0]:
SET SESSION AUTHORIZATION DEFAULT;
drop schema if exists hospital cascade;

In [0]:
DROP RLS POLICY IF EXISTS all_can_see cascade;
DROP RLS POLICY IF EXISTS hide_confidential cascade;
DROP RLS POLICY IF EXISTS only_doctors_can_see cascade;
DROP RLS POLICY IF EXISTS see_only_own_department cascade;

In [0]:
drop table if exists medicine_data cascade;
drop table if exists employees cascade;
drop table if exists patients cascade;
drop table if exists employees_departments cascade;

In [0]:
DROP USER house;
DROP USER cuddy;
DROP USER external;

In [0]:
DROP ROLE  staff; 
DROP ROLE manager; 
DROP ROLE  external;

## Create Tables

In [0]:
drop schema if exists hospital; 
CREATE SCHEMA hospital;

In [0]:
set search_path to hospital;
show search_path;

In [0]:
drop table if exists medicine_data cascade;
CREATE TABLE medicine_data (
    medicine_name      VARCHAR(200),
    medicine_price     INTEGER,
    confidential       BOOLEAN
);

drop table if exists employees cascade;
CREATE TABLE employees (
employee_username  VARCHAR(20),
employee_name      VARCHAR(100),
employee_is_doctor BOOLEAN
);

drop table if exists patients cascade;
CREATE TABLE patients (
patient_dept_id    INTEGER,
patient_name       VARCHAR(100),
patient_birthday   DATE,
patient_medicine       VARCHAR(200),
diagnosis          VARCHAR(200),
confidential       BOOLEAN
);

drop table if exists employees_departments cascade;
CREATE TABLE employees_departments (
employee_username   VARCHAR(20),
department_id       INTEGER 
);

## Load Data

In [0]:
INSERT INTO medicine_data
(medicine_name, medicine_price, confidential)
VALUES
('Ibuprofen', 10, false),
('Cabergolin', 50, true)
;

INSERT INTO employees
(employee_username, employee_name, employee_is_doctor)
VALUES
('house', 'Gregory House', true),
('wilson', 'James Wilson', true),
('cuddy', 'Lisa Cuddy', true),
('janitor', 'Glen Matthews', false),
('external', 'Some Guy', false)
;

INSERT INTO patients
(patient_dept_id, patient_name, patient_birthday, patient_medicine, diagnosis, confidential)
VALUES
(1, 'John Doe Minor', '2008-06-01', 'Ibuprofen', 'some diagnosis', false),
(1, 'Jane Doe Adult', '1992-06-01', 'Cabergolin', 'some other diagnosis', false),
(1, 'Lance Doe Confidential', '1992-06-01', 'Ibuprofen', 'another diagnosis', true),
(2, 'Boris Doe Dept 2', '1992-06-01', 'Cabergolin', 'new  diagnosis', false),
(2, 'Alice Doe Confidential', '1992-06-01', 'Ibuprofen', 'yet another diagnosis', true)
;

INSERT INTO employees_departments
(employee_username, department_id)
VALUES
('house', 1),
('wilson', 2),
('cuddy', 2),
('janitor', 3)
;

## Create Row Level Policy (RLS)

- [RLS aws blog](https://aws.amazon.com/blogs/big-data/achieve-fine-grained-data-security-with-row-level-access-control-in-amazon-redshift/)

In [0]:
CREATE RLS POLICY all_can_see
USING ( true );

CREATE RLS POLICY hide_confidential
WITH ( confidential BOOLEAN )
USING ( confidential = false )
;

-- Note: Employee table is used as lookup in this policy

CREATE RLS POLICY only_doctors_can_see
USING (
    true = (
            SELECT employee_is_doctor
            FROM employees
            WHERE employee_username = current_user
            )
    )
;

GRANT SELECT ON employees
TO RLS POLICY only_doctors_can_see;

CREATE RLS POLICY see_only_own_department
WITH ( patient_dept_id INTEGER )
USING (
    patient_dept_id IN (
                        SELECT department_id
                        FROM employees_departments
                        WHERE employee_username = current_user
                        )
    )
;

GRANT SELECT ON employees_departments 
TO RLS POLICY see_only_own_department;

## Create ROLES

In [0]:
CREATE ROLE staff;
CREATE ROLE manager;
CREATE ROLE external;

## Grant Schema Usage to ROLES

In [0]:
SET SESSION AUTHORIZATION DEFAULT;

grant usage on schema hospital to ROLE manager, ROLE external, ROLE staff;

## Grant Columns Access to ROLES

In [0]:
--- manager can see full table patients and medicine data
GRANT SELECT ON employees, employees_departments, patients, medicine_data TO ROLE manager, ROLE external;

In [0]:
--- staff can see limited columns from medicine data
GRANT SELECT (medicine_name, medicine_price) ON medicine_data 
TO ROLE staff;

In [0]:
--- staff can see, update and delete limited columns from patients
GRANT SELECT (patient_dept_id, patient_name, patient_birthday, patient_medicine, diagnosis) ON patients TO ROLE staff;
GRANT UPDATE (patient_dept_id, patient_name, patient_birthday, patient_medicine, diagnosis) ON patients TO ROLE staff;
GRANT DELETE ON patients TO ROLE staff;

## Atach RLS to ROLES

In [0]:
--- manager can see all medicine data
ATTACH RLS POLICY all_can_see
ON medicine_data
TO ROLE manager;

In [0]:
--- manager can see all patient data
ATTACH RLS POLICY all_can_see
ON patients
TO ROLE manager;

In [0]:
--- staff cannot see confidential medicine data
ATTACH RLS POLICY hide_confidential
ON medicine_data
TO ROLE staff;

--- staff cannot see confidential patient data
ATTACH RLS POLICY hide_confidential
ON patients
TO ROLE staff;

In [0]:
--- only doctors can see patient data
ATTACH RLS POLICY only_doctors_can_see 
ON patients
TO PUBLIC;

In [0]:
--- regular staff (doctors) can see data for patients in their department only
ATTACH RLS POLICY see_only_own_department 
ON patients
TO ROLE staff;

## Enable RLS Permissions

In [0]:
ALTER TABLE medicine_data ROW LEVEL SECURITY on;
ALTER TABLE patients ROW LEVEL SECURITY on;

## Creater USER

In [0]:
CREATE USER house PASSWORD DISABLE;
CREATE USER cuddy PASSWORD DISABLE;
CREATE USER external PASSWORD DISABLE;

GRANT ROLE staff TO house;
GRANT ROLE manager TO cuddy;
GRANT ROLE external TO external;

## Doctor and Manager

In [0]:
--- As Cuddy, who is a doctor and a manager
SET SESSION AUTHORIZATION 'cuddy';

--- policies applied: all_can_see 
SELECT * FROM hospital.medicine_data;

--- policies applied: all_can_see, only_doctors_can_see
SELECT * FROM hospital.patients;

## Staff

In [0]:
--- As House, who is a doctor but not a manager - he is staff in department id 1
SET SESSION AUTHORIZATION 'house';

--- column level access control applied
SELECT * FROM hospital.medicine_data;

--- CLS + RLS policy = hide_confidential
SELECT current_user, medicine_name, medicine_price FROM hospital.medicine_data;

--- column level access control applied 
SELECT * FROM hospital.patients;